In [1]:
# import modules
from jrdb import load
from jrdb import parse
import os
import pandas as pd
from tqdm import tqdm
import numpy as np

In [44]:
df = pd.read_csv('train_data/2022_kyi_sed.csv')

C:\Users\rebor\AppData\Local\Temp\ipykernel_10312\3814963225.py:1: DtypeWarning: Columns (2,6,157) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train_data/2022_kyi_sed.csv')


In [45]:
df

,Unnamed: 0.1,Unnamed: 0,レースキー,場コード,年,回,日,R,馬番,血統登録番号,...,去勢フラグ,乗替フラグ,入厩何走目,入厩年月日,入厩何日前,放牧先,放牧先ランク,厩舎ランク,key,着順
0,0,0,06221101,6,22,1,1,1,1,19102173,...,0.0,1.0,1.0,20211209.0,27,有限会社高橋トレーニングセンター,D,8.0,062211011,11.0
1,1,1,06221101,6,22,1,1,1,2,19104288,...,NaN,NaN,20.0,211126.0,小,野瀬ファームD7,NaN,NaN,062211012,4.0
2,2,2,06221101,6,22,1,1,1,3,19106127,...,NaN,NaN,20.0,2112142.0,2Ｋ,ＳトレーニングセンターC6,NaN,NaN,062211013,16.0
3,3,3,06221101,6,22,1,1,1,4,19103542,...,NaN,NaN,20.0,210813.0,真,歌トレーニングパークD7,NaN,NaN,062211014,3.0
4,4,4,06221101,6,22,1,1,1,5,19101943,...,0.0,9.0,1.0,20211217.0,19,ブルーステーブル,C,8.0,062211015,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46850,46850,46691,9226912,9,22,6,9,12,12,18105223,...,NaN,NaN,20.0,2212022.0,6ノ,ーザンファームしがらきA1,NaN,NaN,922691212,12.0
46851,46851,46692,9226912,9,22,6,9,12,13,17106137,...,0.0,0.0,3.0,20221104.0,NaN,チャンピオンヒルズ,A,3.0,922691213,16.0
46852,46852,46693,9226912,9,22,6,9,12,14,17100461,...,0.0,1.0,1.0,20221210.0,18,吉澤ステーブルＥＡＳＴ,B,3.0,922691214,11.0
46853,46853,46694,9226912,9,22,6,9,12,15,18106389,...,0.0,0.0,1.0,20221209.0,19,チャンピオンヒルズ,A,7.0,922691215,7.0


In [46]:
# 目的変数
df['着順'] = df['着順'].apply(lambda x: 1 if x <= 3 else 0)

In [48]:
# オブジェクト型の列の名前を取得
non_numeric_columns = df.select_dtypes(include=['object']).columns

# 日付の列を除いたIndexを作成
non_numeric_columns = non_numeric_columns[(non_numeric_columns != '日') & (non_numeric_columns != '回')]

non_numeric_columns

Index(['レースキー', '馬名', '騎手名', '調教師名', '調教師所属', '前走1レースキー', '前走2レースキー',
       '前走3レースキー', '前走4レースキー', '前走5レースキー', 'ペース予想', '馬主名', '馬特記3', '万券印',
       '激走タイプ', '入厩何日前', '放牧先', '放牧先ランク', 'key'],
      dtype='object')

In [49]:
# str型に変換
df[non_numeric_columns] = df[non_numeric_columns].astype(str)

from sklearn.preprocessing import LabelEncoder
# LabelEncoderオブジェクトを作成
le = LabelEncoder()

# 選択された非数値列ごとにラベルエンコーディングを適用
df[non_numeric_columns] = df[non_numeric_columns].apply(lambda x: le.fit_transform(x))

# エンコーディング後のデータフレームを表示
df[non_numeric_columns].head()

,レースキー,馬名,騎手名,調教師名,調教師所属,前走1レースキー,前走2レースキー,前走3レースキー,前走4レースキー,前走5レースキー,ペース予想,馬主名,馬特記3,万券印,激走タイプ,入厩何日前,放牧先,放牧先ランク,key
0,647,3506,49,80,11,1439,1680,6493,6838,7127,0,315,240,10,16,149,744,3,8980
1,647,7597,12,200,11,2214,2460,1151,6838,7127,0,146,235,10,16,768,1063,5,8988
2,647,3156,74,129,11,2104,1032,6493,6838,7127,0,112,4,10,2,211,1166,5,8989
3,647,8879,91,125,11,2202,1797,1890,1943,2924,0,58,155,10,16,789,852,5,8990
4,647,2103,102,141,11,1397,1641,2627,632,7127,0,386,240,5,14,80,301,2,8991


In [50]:
df

,Unnamed: 0.1,Unnamed: 0,レースキー,場コード,年,回,日,R,馬番,血統登録番号,...,去勢フラグ,乗替フラグ,入厩何走目,入厩年月日,入厩何日前,放牧先,放牧先ランク,厩舎ランク,key,着順
0,0,0,647,6,22,1,1,1,1,19102173,...,0.0,1.0,1.0,20211209.0,149,744,3,8.0,8980,0
1,1,1,647,6,22,1,1,1,2,19104288,...,NaN,NaN,20.0,211126.0,768,1063,5,NaN,8988,0
2,2,2,647,6,22,1,1,1,3,19106127,...,NaN,NaN,20.0,2112142.0,211,1166,5,NaN,8989,0
3,3,3,647,6,22,1,1,1,4,19103542,...,NaN,NaN,20.0,210813.0,789,852,5,NaN,8990,1
4,4,4,647,6,22,1,1,1,5,19101943,...,0.0,9.0,1.0,20211217.0,80,301,2,8.0,8991,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46850,46850,46691,3422,9,22,6,9,12,12,18105223,...,NaN,NaN,20.0,2212022.0,458,476,5,NaN,46683,0
46851,46851,46692,3422,9,22,6,9,12,13,17106137,...,0.0,0.0,3.0,20221104.0,722,217,0,3.0,46684,0
46852,46852,46693,3422,9,22,6,9,12,14,17100461,...,0.0,1.0,1.0,20221210.0,78,608,1,3.0,46685,0
46853,46853,46694,3422,9,22,6,9,12,15,18106389,...,0.0,0.0,1.0,20221209.0,80,217,0,7.0,46686,0


In [28]:
df['年月日'] = pd.to_datetime(df['年月日'])
df = df.sort_values(by='年月日')
train_df = df.iloc[:int(len(df) * 0.8)]
test_df = df.iloc[int(len(df) * 0.8):]

KeyError: '年月日'

In [ ]:
# 不要なカラムの削除
# '着順'と'年月日'カラムを削除
train_df = train_df.drop(['着順', '年月日'], axis=1)
test_df = test_df.drop(['着順', '年月日'], axis=1)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import f1_score, roc_auc_score

# データの分割
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['target'], axis=1), train_SED_df['target'], test_size=0.2, random_state=42)


# LightGBMのパラメータ設定
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_child_samples': 20,
    'max_bin': 255,
    'verbose': -1
}

# モデルのトレーニング
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

gbm = lgb.train(params, lgb_train, num_boost_round=500, valid_sets=lgb_eval, early_stopping_rounds=20)

# モデルの評価
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# 予測値を二値に変換
y_pred_binary = (y_pred >= 0.5).astype(int)
f1 = f1_score(y_test, y_pred_binary)
auc = roc_auc_score(y_test, y_pred)

# 評価指標の表示
print(f'F1 Score: {f1}')
print(f'AUC Score: {auc}')

In [ ]:
# Feature Importanceの表示
importance = pd.DataFrame(gbm.feature_importance(), index=X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
importance.head(20)